In [1]:
!mkdir ../checkpoints
!wget https://openmmlab.oss-accelerate.aliyuncs.com/mmsegmentation/v0.5/deeplabv3/deeplabv3_r50-d8_512x1024_40k_cityscapes/deeplabv3_r50-d8_512x1024_40k_cityscapes_20200605_022449-acadc2f8.pth -P ../checkpoints

mkdir: cannot create directory ‘../checkpoints’: File exists
--2020-07-07 08:54:25--  https://open-mmlab.s3.ap-northeast-2.amazonaws.com/mmsegmentation/models/pspnet/pspnet_r50-d8_512x1024_40k_cityscapes/pspnet_r50-d8_512x1024_40k_cityscapes_20200605_003338-2966598c.pth
Resolving open-mmlab.s3.ap-northeast-2.amazonaws.com (open-mmlab.s3.ap-northeast-2.amazonaws.com)... 52.219.58.55
Connecting to open-mmlab.s3.ap-northeast-2.amazonaws.com (open-mmlab.s3.ap-northeast-2.amazonaws.com)|52.219.58.55|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 196205945 (187M) [application/x-www-form-urlencoded]
Saving to: ‘../checkpoints/pspnet_r50-d8_512x1024_40k_cityscapes_20200605_003338-2966598c.pth.1’

pspnet_r50-d8_512x1 100%[===================>] 187.12M  16.5MB/s    in 13s     

2020-07-07 08:54:38 (14.8 MB/s) - ‘../checkpoints/pspnet_r50-d8_512x1024_40k_cityscapes_20200605_003338-2966598c.pth.1’ saved [196205945/196205945]



In [1]:
from mmseg.apis import init_segmentor, inference_segmentor, show_result_pyplot
from mmseg.core.evaluation import get_palette
import time

In [2]:
config_file = '/opt/ml/lab/mmsegmentation/configs/sm_config/icnet_r18-d8_832x832_80k_cityscapes.py'
checkpoint_file = '/opt/ml/lab/mmsegmentation/work_dirs/icnet_r18-d8_832x832_80k_cityscapes/best_mIoU_epoch_115.pth'

# config_file = '/opt/ml/lab/mmsegmentation/configs/pspnet/pspnet_r18-d8_769x769_80k_cityscapes.py'
# checkpoint_file = '/opt/ml/lab/mmsegmentation/pretrain/pspnet_r18-d8_769x769_80k_cityscapes_20201225_021458-3deefc62.pth'

In [3]:
# build the model from a config file and a checkpoint file
model = init_segmentor(config_file, checkpoint_file, device='cpu')

load checkpoint from local path: /opt/ml/lab/mmsegmentation/work_dirs/icnet_r18-d8_832x832_80k_cityscapes/best_mIoU_epoch_276.pth


In [6]:
# icnet
from torch.autograd import profiler
import torch
img = '/opt/ml/test_images.jpeg'
g_start = torch.cuda.Event(enable_timing=True)
g_end = torch.cuda.Event(enable_timing=True)
g_start.record()
c_start = time.time()
result = inference_segmentor(model, img)
c_end = time.time()
g_end.record()

g_inference = g_start.elapsed_time(g_end)/1000
print(f'cpu time : {(c_end-c_start)}\t using torch.cuda.Event : {g_inference}')
print(f'fps per frame')
print(result)

/opt/conda/envs/mmseg/lib/python3.7/site-packages/torch/nn/functional.py:3000: UserWarning: The default behavior for interpolate/upsample with float scale_factor changed in 1.6.0 to align with other frameworks/libraries, and uses scale_factor directly, instead of relying on the computed output size. If you wish to keep the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/upsample with float scale_factor changed "
cpu time : 1.247389554977417	 using torch.cuda.Event : 1.2476231689453126
fps per frame
[array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]])]


In [5]:
# pspnet
from torch.autograd import profiler
import torch
img = '/opt/ml/test_images.jpeg'
g_start = torch.cuda.Event(enable_timing=True)
g_end = torch.cuda.Event(enable_timing=True)
g_start.record()
c_start = time.time()
result = inference_segmentor(model, img)
c_end = time.time()
g_end.record()

g_inference = g_start.elapsed_time(g_end)/1000
print(f'cpu time : {(c_end-c_start)}\t using torch.cuda.Event : {g_inference}')

/opt/ml/lab/mmsegmentation/mmseg/ops/wrappers.py:23: UserWarning: When align_corners=True, the output would more aligned if input size (6, 6) is `x+1` and out size (97, 97) is `nx+1`
  f'When align_corners={align_corners}, '
cpu time : 14.864436864852905	 using torch.cuda.Event : 14.8650390625


In [5]:
# show the results
show_result_pyplot(model, img, result, get_palette('cityscapes'))

AssertionError: 